In [8]:
import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/arousal/train'
validation_dir_name = '../../../data/classification/arousal/validation'
test_dir_name = '../../../data/classification/arousal/test'

# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

In [9]:
import sys
import keras
print('Keras version:',keras.__version__)

from keras import backend as K

Keras version: 2.3.1


In [10]:
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)

In [11]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [12]:
def create_new_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=train_generator.image_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2))

    model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=train_generator.image_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2))

    model.add(Flatten())

    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.2))
    
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    opt = optimizers.Adam(lr=0.001)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [13]:
kf = KFold(n_splits=5, shuffle=True)
accuracies = []

In [14]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=200
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')
    
    model = create_new_model()
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    loss, acc = model.evaluate_generator(validation_generator)
    accuracies.append(acc)
    print(acc)

Found 6752 images belonging to 2 classes.
Found 1688 images belonging to 2 classes.
Epoch 1/200
27/27 [==============================] - 20s 729ms/step - loss: 0.6900 - accuracy: 0.5650
Epoch 2/200
27/27 [==============================] - 8s 292ms/step - loss: 0.6797 - accuracy: 0.5844
Epoch 3/200
27/27 [==============================] - 8s 283ms/step - loss: 0.6802 - accuracy: 0.5844
Epoch 4/200
27/27 [==============================] - 8s 279ms/step - loss: 0.6790 - accuracy: 0.5844
Epoch 5/200
27/27 [==============================] - 8s 287ms/step - loss: 0.6777 - accuracy: 0.5844
Epoch 6/200
27/27 [==============================] - 8s 288ms/step - loss: 0.6774 - accuracy: 0.5844
Epoch 7/200
27/27 [==============================] - 7s 271ms/step - loss: 0.6782 - accuracy: 0.5840
Epoch 8/200
27/27 [==============================] - 7s 263ms/step - loss: 0.6763 - accuracy: 0.5850
Epoch 9/200
27/27 [==============================] - 7s 254ms/step - loss: 0.6746 - accuracy: 0.5877
Epoch 

27/27 [==============================] - 7s 268ms/step - loss: 0.2838 - accuracy: 0.8722
Epoch 81/200
27/27 [==============================] - 7s 256ms/step - loss: 0.2821 - accuracy: 0.8741
Epoch 82/200
27/27 [==============================] - 7s 257ms/step - loss: 0.2774 - accuracy: 0.8729
Epoch 83/200
27/27 [==============================] - 7s 255ms/step - loss: 0.2610 - accuracy: 0.8846
Epoch 84/200
27/27 [==============================] - 7s 261ms/step - loss: 0.2618 - accuracy: 0.8834
Epoch 85/200
27/27 [==============================] - 7s 274ms/step - loss: 0.2595 - accuracy: 0.8858
Epoch 86/200
27/27 [==============================] - 7s 262ms/step - loss: 0.2520 - accuracy: 0.8879
Epoch 87/200
27/27 [==============================] - 7s 255ms/step - loss: 0.2544 - accuracy: 0.8919
Epoch 88/200
27/27 [==============================] - 7s 257ms/step - loss: 0.2554 - accuracy: 0.8898
Epoch 89/200
27/27 [==============================] - 7s 264ms/step - loss: 0.2499 - accuracy: 

27/27 [==============================] - 7s 263ms/step - loss: 0.1209 - accuracy: 0.9519
Epoch 160/200
27/27 [==============================] - 7s 278ms/step - loss: 0.1170 - accuracy: 0.9529
Epoch 161/200
27/27 [==============================] - 7s 273ms/step - loss: 0.1294 - accuracy: 0.9492
Epoch 162/200
27/27 [==============================] - 7s 275ms/step - loss: 0.1240 - accuracy: 0.9485
Epoch 163/200
27/27 [==============================] - 7s 270ms/step - loss: 0.1200 - accuracy: 0.9528
Epoch 164/200
27/27 [==============================] - 8s 283ms/step - loss: 0.1206 - accuracy: 0.9511
Epoch 165/200
27/27 [==============================] - 8s 278ms/step - loss: 0.1229 - accuracy: 0.9477
Epoch 166/200
27/27 [==============================] - 7s 273ms/step - loss: 0.1166 - accuracy: 0.9505
Epoch 167/200
27/27 [==============================] - 8s 283ms/step - loss: 0.1177 - accuracy: 0.9526
Epoch 168/200
27/27 [==============================] - 8s 278ms/step - loss: 0.1168 - a

27/27 [==============================] - 7s 270ms/step - loss: 0.5127 - accuracy: 0.7361
Epoch 39/200
27/27 [==============================] - 7s 265ms/step - loss: 0.5052 - accuracy: 0.7377
Epoch 40/200
27/27 [==============================] - 7s 270ms/step - loss: 0.5016 - accuracy: 0.7393
Epoch 41/200
27/27 [==============================] - 8s 280ms/step - loss: 0.4862 - accuracy: 0.7527
Epoch 42/200
27/27 [==============================] - 8s 284ms/step - loss: 0.4836 - accuracy: 0.7519
Epoch 43/200
27/27 [==============================] - 7s 268ms/step - loss: 0.4821 - accuracy: 0.7540
Epoch 44/200
27/27 [==============================] - 8s 300ms/step - loss: 0.4689 - accuracy: 0.7633
Epoch 45/200
27/27 [==============================] - 8s 284ms/step - loss: 0.4631 - accuracy: 0.7701
Epoch 46/200
27/27 [==============================] - 8s 287ms/step - loss: 0.4597 - accuracy: 0.7670
Epoch 47/200
27/27 [==============================] - 7s 272ms/step - loss: 0.4481 - accuracy: 

27/27 [==============================] - 8s 281ms/step - loss: 0.1754 - accuracy: 0.9258
Epoch 119/200
27/27 [==============================] - 7s 253ms/step - loss: 0.1648 - accuracy: 0.9338
Epoch 120/200
27/27 [==============================] - 7s 249ms/step - loss: 0.1704 - accuracy: 0.9285
Epoch 121/200
27/27 [==============================] - 7s 248ms/step - loss: 0.1664 - accuracy: 0.9289
Epoch 122/200
27/27 [==============================] - 7s 246ms/step - loss: 0.1562 - accuracy: 0.9339
Epoch 123/200
27/27 [==============================] - 7s 243ms/step - loss: 0.1565 - accuracy: 0.9351
Epoch 124/200
27/27 [==============================] - 7s 251ms/step - loss: 0.1524 - accuracy: 0.9379
Epoch 125/200
27/27 [==============================] - 7s 253ms/step - loss: 0.1488 - accuracy: 0.9351
Epoch 126/200
27/27 [==============================] - 7s 272ms/step - loss: 0.1457 - accuracy: 0.9388
Epoch 127/200
27/27 [==============================] - 7s 264ms/step - loss: 0.1484 - a

...

27/27 [==============================] - 9s 317ms/step - loss: 0.6138 - accuracy: 0.6488
Epoch 35/200
27/27 [==============================] - 8s 301ms/step - loss: 0.6070 - accuracy: 0.6631
Epoch 36/200
27/27 [==============================] - 8s 294ms/step - loss: 0.5970 - accuracy: 0.6700
Epoch 37/200
27/27 [==============================] - 9s 317ms/step - loss: 0.5890 - accuracy: 0.6770
Epoch 38/200
27/27 [==============================] - 8s 306ms/step - loss: 0.5877 - accuracy: 0.6782
Epoch 39/200
27/27 [==============================] - 8s 281ms/step - loss: 0.5857 - accuracy: 0.6811
Epoch 40/200
27/27 [==============================] - 8s 296ms/step - loss: 0.5725 - accuracy: 0.6871
Epoch 41/200
27/27 [==============================] - 8s 294ms/step - loss: 0.5619 - accuracy: 0.7042
Epoch 42/200
27/27 [==============================] - 8s 311ms/step - loss: 0.5613 - accuracy: 0.6956
Epoch 43/200
27/27 [==============================] - 8s 300ms/step - loss: 0.5592 - accuracy: 

27/27 [==============================] - 9s 316ms/step - loss: 0.2101 - accuracy: 0.9102
Epoch 115/200
27/27 [==============================] - 8s 307ms/step - loss: 0.2029 - accuracy: 0.9101
Epoch 116/200
27/27 [==============================] - 8s 280ms/step - loss: 0.2080 - accuracy: 0.9079
Epoch 117/200
27/27 [==============================] - 7s 272ms/step - loss: 0.2034 - accuracy: 0.9148
Epoch 118/200
27/27 [==============================] - 7s 253ms/step - loss: 0.1931 - accuracy: 0.9151
Epoch 119/200
27/27 [==============================] - 8s 279ms/step - loss: 0.1943 - accuracy: 0.9197
Epoch 120/200
27/27 [==============================] - 7s 263ms/step - loss: 0.1959 - accuracy: 0.9169
Epoch 121/200
27/27 [==============================] - 7s 254ms/step - loss: 0.2027 - accuracy: 0.9140
Epoch 122/200
27/27 [==============================] - 7s 250ms/step - loss: 0.1882 - accuracy: 0.9168
Epoch 123/200
27/27 [==============================] - 7s 257ms/step - loss: 0.1850 - a

27/27 [==============================] - 7s 249ms/step - loss: 0.1073 - accuracy: 0.9559
Epoch 194/200
27/27 [==============================] - 7s 249ms/step - loss: 0.1074 - accuracy: 0.9557
Epoch 195/200
27/27 [==============================] - 7s 250ms/step - loss: 0.1009 - accuracy: 0.9596
Epoch 196/200
27/27 [==============================] - 7s 246ms/step - loss: 0.0970 - accuracy: 0.9615
Epoch 197/200
27/27 [==============================] - 7s 249ms/step - loss: 0.0924 - accuracy: 0.9640
Epoch 198/200
27/27 [==============================] - 7s 250ms/step - loss: 0.0949 - accuracy: 0.9615
Epoch 199/200
27/27 [==============================] - 7s 252ms/step - loss: 0.1021 - accuracy: 0.9613
Epoch 200/200
27/27 [==============================] - 7s 249ms/step - loss: 0.1047 - accuracy: 0.9581
0.7606635093688965
Found 6752 images belonging to 2 classes.
Found 1688 images belonging to 2 classes.
Epoch 1/200
27/27 [==============================] - 7s 271ms/step - loss: 0.7050 - acc

27/27 [==============================] - 7s 249ms/step - loss: 0.3405 - accuracy: 0.8418
Epoch 73/200
27/27 [==============================] - 7s 251ms/step - loss: 0.3451 - accuracy: 0.8405
Epoch 74/200
27/27 [==============================] - 7s 250ms/step - loss: 0.3298 - accuracy: 0.8430
Epoch 75/200
27/27 [==============================] - 7s 255ms/step - loss: 0.3310 - accuracy: 0.8430
Epoch 76/200
27/27 [==============================] - 7s 255ms/step - loss: 0.3297 - accuracy: 0.8491
Epoch 77/200
27/27 [==============================] - 7s 251ms/step - loss: 0.3202 - accuracy: 0.8520
Epoch 78/200
27/27 [==============================] - 7s 256ms/step - loss: 0.3173 - accuracy: 0.8537
Epoch 79/200
27/27 [==============================] - 7s 247ms/step - loss: 0.3090 - accuracy: 0.8529
Epoch 80/200
27/27 [==============================] - 7s 255ms/step - loss: 0.3176 - accuracy: 0.8550
Epoch 81/200
27/27 [==============================] - 7s 247ms/step - loss: 0.3093 - accuracy: 

Epoch 152/200
27/27 [==============================] - 7s 252ms/step - loss: 0.1421 - accuracy: 0.9416
Epoch 153/200
27/27 [==============================] - 7s 254ms/step - loss: 0.1393 - accuracy: 0.9439
Epoch 154/200
27/27 [==============================] - 7s 255ms/step - loss: 0.1470 - accuracy: 0.9418
Epoch 155/200
27/27 [==============================] - 4s 162ms/step - loss: 0.1403 - accuracy: 0.9422
Epoch 156/200
27/27 [==============================] - 4s 135ms/step - loss: 0.1357 - accuracy: 0.9433
Epoch 157/200
27/27 [==============================] - 4s 136ms/step - loss: 0.1355 - accuracy: 0.9434
Epoch 158/200
27/27 [==============================] - 4s 134ms/step - loss: 0.1353 - accuracy: 0.9452
Epoch 159/200
27/27 [==============================] - 4s 136ms/step - loss: 0.1296 - accuracy: 0.9492
Epoch 160/200
27/27 [==============================] - 4s 136ms/step - loss: 0.1279 - accuracy: 0.9483
Epoch 161/200
27/27 [==============================] - 4s 135ms/step - lo

In [15]:
np.average(accuracies)

0.7543838858604431